In [35]:
# Import necessary libraries
import os
import cv2
import numpy as np
from pycocotools.coco import COCO
from scipy.ndimage import zoom

In [36]:
# Load COCO annotations (Assuking we have COCO type annotations in a JSON file)
coco = COCO("../damage_ratio_calc_data/COCO annotations/instances_default.json")
image_ids = coco.getImgIds()
print(f"Image IDs are: {image_ids}")

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Image IDs are: [1, 2, 3]


In [37]:
# Output directory stuff
output_dir = "../damage_ratio_calc_data/segmented_objects"
os.makedirs(output_dir, exist_ok=True)

In [49]:
for image_id in image_ids:
    # Load image information
    image_info = coco.loadImgs(image_id)[0]
    image = cv2.imread(f"../damage_ratio_calc_data/damage_ratio_images/{image_info["file_name"]}")  # Read the image

    # Check if the image is loaded properly
    if image is None:
        print(f"Error: Unable to load image {image_info['file_name']}")
        continue

    image_name = os.path.splitext(os.path.basename(image_info["file_name"]))[0]  # Get name without extension

    # Create folder for this image
    image_folder = os.path.join(output_dir, image_name)
    os.makedirs(image_folder, exist_ok=True)

    # Get annotations for this image
    ann_ids = coco.getAnnIds(imgIds=image_id)
    anns = coco.loadAnns(ann_ids)

    for idx, ann in enumerate(anns):
        # Generate the segmentation mask for the current annotation
        mask = coco.annToMask(ann)

        # Check if the mask is valid
        if mask is None or mask.sum() == 0:
            print(f"Warning: Invalid or empty mask for annotation {ann['id']} in image {image_name}")
            continue

        bbox = ann['bbox']  # [x, y, width, height]
        x_min, y_min, width, height = bbox
        buffer = 10.0
        # print(bbox)
        # print(int(y_min - buffer),int(y_min + height + buffer), int(x_min - buffer), int(x_min + width + buffer))

        # Create a mask that includes only the segmented area
        mask_region = mask[int(y_min - buffer):int(y_min + height + buffer), int(x_min - buffer):int(x_min + width + buffer)]

        # Create a new black image which is the same size as the bbox for the segmented object
        # object_img = np.zeros_like(image, dtype=np.uint8)
        # print(object_img.shape)
        cropped_image = image[int(y_min - buffer):int(y_min + height + buffer), int(x_min - buffer):int(x_min + width + buffer)]
        object_img = np.zeros_like(cropped_image, dtype=np.uint8)
        # print(object_img.shape)

        # Get the region of interest from the original image
        image_region = image[int(y_min - buffer):int(y_min + height + buffer), int(x_min - buffer):int(x_min + width + buffer)]

        # Place the segmented object on the black background using the mask
        # object_img[int(y_min - buffer):int(y_min + height + buffer), int(x_min - buffer):int(x_min + width + buffer)][mask_region > 0] = image_region[mask_region > 0]
        object_img[mask_region > 0] = image_region[mask_region > 0]

        # Save the extracted object as a separate PNG image
        object_path = os.path.join(image_folder, f"{image_name}_object_{idx}.png")
        cv2.imwrite(object_path, object_img)

    print(f"Processed: {image_name}")

Processed: 183_jpg.rf.c3a79aad316f75fe23536c3bbbd6da51
Processed: 184_jpg.rf.9edad8ca8d25dc9949b48968cca6e41c
Processed: 191_jpg.rf.e27c030e763e58ce48964e670158b6e7
